In [1]:
import pandas as pd
import random
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [2]:
df = pd.read_csv('query_file.csv')
df.head()


,Label,Query
0,abruzzo,abruzzo
1,abruzzo,abruzzo accommodation
2,abruzzo,abruzzo attractions
3,abruzzo,abruzzo calgary
4,abruzzo,abruzzo club


In [3]:
df['category_id'] = df['Label'].factorize()[0]
category_id_df = df[['Label', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Label']].values)
df.head()

,Label,Query,category_id
0,abruzzo,abruzzo,0
1,abruzzo,abruzzo accommodation,0
2,abruzzo,abruzzo attractions,0
3,abruzzo,abruzzo calgary,0
4,abruzzo,abruzzo club,0


In [4]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english',max_features = 5000)
tfidf.fit(df.Query)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='latin-1',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=5, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [8]:
features = tfidf.fit_transform(df.Query)
labels = df.category_id
features.shape

(1104908, 5000)

In [9]:
from sklearn.feature_selection import chi2
import numpy as np

In [22]:
features

<1104908x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 3177083 stored elements in Compressed Sparse Row format>

In [51]:
import csv


In [55]:
with open('correlations.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Category ID", "Label", "Most Correlated", "Least Correlated"])
    for Label, category_id in sorted(category_to_id.items()):
        features_chi2 = chi2(features, labels == category_id)
        indices = np.argsort(features_chi2[0])
        feature_names = np.array(tfidf.get_feature_names())[indices]
        
        writer.writerow([category_id, Label, str(feature_names[-1]), str(feature_names[0])])

#         print("# '{}':".format(Label))
#         print("  . Most correlated:\n. {}".format('\n. '.join(feature_names[-N:])))
#         print("  . Least correlated:\n. {}".format('\n. '.join(feature_names[0:N])))